# 換気量計算モジュール

## $\S1$：換気量計算における基礎式

建物内の通気量の計算方法には以下の2つの方法がある。
- ①通気率$Q_0$および隙間特性値$n$をもとにした計算。
- ②相当隙間面積$\alpha A$をもとにした計算。  

①はJISやISOにおける気密測定において用いられる式である。  
また、推測にはなるが②はベルヌーイの式より算出された理論式であり、①は実際の隙間において生じる開口部との摩擦損損失を考慮するための実験式であると推測される。  
鳥海ら(2016)によると、①式はクラックや隙間のような狭い開口に用いられるべき乗則であり、②式は大きな開口に用いられる二乗則であるとされている。

②は①における隙間特性値$n=2$と固定した場合の式に等しく、開口部の面積$A$が分かれば未知パラメーターが流量係数$\alpha$のみとなるため、与える隙間条件が単純となり、隙間の特性も感覚的にわかりやすくなる。両式の差については宮坂ら(2000)により検討がなされており、住宅内における換気計算においては差が小さいことが示されている。

【参考文献】
- エース建築環境工学II 13.2章 式(13.8)から(13.27)
- 宮坂ら(2001)建築学会近畿支部研究報告集
- 藤田・松下(2014)黄表紙
- 鳥海ら(2016), 「集合住宅における隙間の評価法に関する研究」, 日本建築学会環境系論文集  
など


<img src="picture/換気量の計算式.png" width="70%">

なお、換気量の計算を考える際にはベルヌーイの式が重要となる。
ベルヌーイの式は力学的エネルギー保存の法則を圧力の形で表現したものである。  
特に、ダクト内を通る空気の圧力損失等を考慮すれば以下のように表すことができる。

$$ p_1 + \frac{1}{{2}}\rho v_1^2 + \rho g h_1 = p_2 + \frac{1}{{2}}\rho v_2^2 + \rho g h_2 + \Delta p_e $$  

また、圧力損失が無く、圧力差により空気が駆動されると仮定すれば与式は以下のようになる。
$$ p_1 - p_2 = \frac{1}{{2}}\rho v^2 $$  

従って、圧力差による空気の流れは以下のように与えられる。  
$$ v = \sqrt{\frac{2}{\rho}} (\Delta p)^{\frac{1}{2}}$$  


### $\S 1.1$ 換気回数を用いた基礎式

$$ Q = \frac{1}{3600}\frac{V}{n} $$  

- $Q$：換気量[m<sup>3</sup>/s]   
- $V$：室容積[m<sup>3</sup>]  
- $n：換気回数[1/h]  

※換気回数は通常1時間当たりの空気の交換回数として表される。

In [1]:
cal_air_flux_by_VR( n::Float64, V::Float64 ) = 1.0 / 3600.0 * (dp)^(1/n)
cal_air_flux_by_VR(;n::Float64, V::Float64 ) = cal_air_flux_by_VR( n, V )

cal_air_flux_by_VR (generic function with 2 methods)

### $\S 1.2$ 隙間を通じた換気（漏気）に関する基礎式

$$ Q = k^{'} \Delta p^{\frac{1}{n}} $$  

- $Q$：空気流量（または通気量）[m<sup>3</sup>/s]または[m<sup>3</sup>/h]   
- $k^{'}$：すきまの特性値（差圧1Pa時における通気量）[m<sup>3</sup>/sPa]または[m<sup>3</sup>/hPa]  
- $\Delta p$：室間の差圧[Pa]  
- $n$：隙間の特性値(1～2)

In [2]:
cal_air_flux_by_leakage( k::Float64, dp::Float64, n::Float64 ) = k * (dp)^(1/n)
cal_air_flux_by_leakage(;k::Float64, dp::Float64, n::Float64 ) = cal_air_flux_by_leakage( k, dp, n )

cal_air_flux_by_leakage (generic function with 2 methods)

圧力差に対する依存性  

$$ \frac{\partial Q}{\partial p} = \frac{1}{n} k^{'} (\Delta p)^{\frac{1-n}{n}} $$

In [3]:
cal_dQdp_by_leakage( k::Float64, dp::Float64, n::Float64 ) = k / n * dp ^ ((1-n)/n)
cal_dQdp_by_leakage(;k::Float64, dp::Float64, n::Float64 ) = cal_dQdp( k, dp, n ) 

cal_dQdp_by_leakage (generic function with 2 methods)

実計算上における通気量・隙間特性値の取り扱いについて示す。  
建物の通気量・隙間特性値を開口部ごとに測定することは現実的ではない。  
そこで、単位長さあたりの通気量$q_0$を用い、通気量を以下のように与える。

$$ Q = q_0 L (\Delta p)^{1/n} $$  

- $q_0$：単位長さあたりの通気量[m<sup>2</sup>/ms]または[m<sup>2</sup>/mh]
- $L$：隙間の長さ[m]

このようにすることで、開口部の隙間特性に関する文献値を参照することができる。  
なお、宮坂(2002)修士論文により開口部の隙間特性値の文献値がまとめられている。
<img src="picture/隙間特性値一覧_宮坂.png" width="70%">

### $\S 1.3$ 有効開口面積$\alpha A$を用いた計算式

ベルヌーイの式をもとに空気の流速に（有効開口）面積を乗じることで開口部における流量を計算することができる。

$$ 
    Q = \alpha A \sqrt{\frac{2}{\rho}}(\Delta p)^{\frac{1}{2}} \
$$  

- $\alpha$：流量係数   
- $A$：隙間面積[m<sup>2</sup>]
- $\rho$：空気の密度[kg/m<sup>3</sup>]
- $\Delta p$：室間の差圧[Pa]  

※$\alpha A$を相当隙間面積あるいは有効開口面積と呼ぶ。

In [4]:
cal_air_flux_by_opening( alpha::Float64, A::Float64, rho::Float64, dp::Float64 ) = alpha * A * (2/rho)^(1/2) * dp ^ (1/2)
cal_air_flux_by_opening(;alpha::Float64, A::Float64, rho::Float64, dp::Float64 ) = cal_air_flux_by_opening( alpha, A, rho, dp )

cal_air_flux_by_opening (generic function with 2 methods)

圧力差に対する依存性  

$$ \frac{\partial Q}{\partial p} = \frac{1}{2} \alpha A \sqrt{\frac{2}{\rho}} (\Delta p)^{-\frac{1}{2}} $$

### $\S 1.4$ 流量係数と隙間特性値の関係  

【参考文献】
- 宮坂ら(2001)建築学会近畿支部研究報告集

隙間特性値$n=2$とすると、$\S 3.2$と$\S 3.1$の式は等しくなる。  
従って、$\S 3.2$に示すような建築物における一般的な開口においても換気量は以下のように記述することが出来る。

$Q = Q_o * (\Delta p)^{1/2} = \alpha A \sqrt{ \frac{2}{\rho} } (\Delta p)^{1/2}$  

よって流れに対して有効な開口面積$\alpha A$（以下隙間相当面積）と隙間特性値$Q_o, n$には以下の関係が成り立つ。  

$\alpha A = \sqrt{\frac{\rho}{2}} Q_o (p)^{\frac{1}{n} - \frac{1}{2} }$

この式より隙間相当面積$\alpha A$は圧力差$p$によって変化することが分かる。  
そのため、この式を用いる場合、基準圧力差に対する隙間特性値$\alpha A$を示すことが一般的であり、基準圧力差として日本では9.8Pa、カナダでは10.0Pa、アメリカでは4.0Paが用いられる。

In [ ]:
function cal_effective_opening_area( rho, Qo, dp, n )
    return ( rho / 2.0 )^(1/2) * Qo * dp ^( 1/n - 1/2 )
end

cal_effective_opening_area (generic function with 1 method)

この差はnが小さくなるほど（クラックのような隙間になるほど）大きくなる。  
一方、宮坂ら(2001)は隙間相当面積が一定値で与えられると換気計算において取り扱いが容易になることから、  
$n=2$とし、  

$\alpha A = \sqrt{\frac{\rho}{2}} Q_o$  

として隙間相当面積を求めたうえで計算を行った場合と、  
$n≠2$とし$Q_o$と$n$を用いた方法との比較を行い、差異がほとんど無いことを検証している。

## $\S 4$ 実際の建物開口における換気・流量計数の取り扱い方

【参考文献】
- エース建築環境工学II 13.2章 式(13.26),(13.27)

#### $\S 3.5.1 $ 数値計算における基礎式

流量計算は小開口または隙間を持つ場合とクラックで計算方法が幾分か異なる。  
例えば、前者は開口の面積や幅といった空間パラメータが必要になる一方で、クラックの場合は壁全体から換気が行われるためこれらのパラメータが不必要となる。  

ここで本プログラムでは流量の計算を統一するするため下記の関数を用いている。

流量の計算ではクラック法を用いて以下の式を用いられる。  
このとき、qは単位体積長さあたりの通気量であり$q = Q/L$[m3/s m]とすると、

基礎式：（修正）  
$$
    Q = Q_0 (\Delta p)^{1/n} = q_0 L (\Delta p)^{1/n}
$$  

従って、$ Q_0 = q_0 L $である。   
なお、aは経験常数であり、指数n（隙間特性値）は1~2の値をとる。  

流量を計算するモジュール内ではこの式を基本とした計算がなされている。

従って、隙間特性値$n$および経験常数$a$が必要となる。  
隙間特性値$n$は通常入力パラメータとして与えるため、以下では経験常数$a$を求める方法について示す。


### $\S 3.5.2 $ 入力条件の違いによる経験常数$a$の決定方法  


#### 1. 入力値：隙間相当面積

前述のように隙間相当面積が一定値として扱えるものと仮定すれば n = 2より、  

$$
    q_0 = \frac{Q_o}{L} =  \frac{\alpha A}{L} \sqrt{\frac{2}{\rho}} 
$$  

経験常数は上記のように与えられる。

In [ ]:
function cal_a_by_effective_opening_area( alpha, A, L, rho )
    return alpha * A / L * ( 2.0 / rho )^2
end

cal_a_by_effective_opening_area (generic function with 1 method)

#### 2. 入力値：基準圧力差(9.8Pa)あたりの換気量  

入力値として基準圧力差(9.8Pa)あたりの換気量が与えられているとする。  
このときの換気量は以下のように表される。

$Q_{9.8Pa}^{'} = Q_o * (9.8[Pa])^{\frac{1}{n}}$  

従って、1Paあたりの換気量は、

$Q_o = Q_{9.8Pa}^{'} * (\frac{1}{9.8[Pa]})$  

なお、換気量として[m3/h]が用いられることも多いため、これを[m3/s]に変換することも必要である。


1Pa時の流量とするとp = 1であり、  

$ ⇒　QV = \alpha SV \sqrt{ \frac{2}{\rho} }$  
$ ⇒　QH = \alpha SH \sqrt{ \frac{2}{\rho} }$  

- AI：流量係数[-] ※AIと書いた場合、特に初期条件を決定するための流量計数alpha initial?を意味するとのこと
- QV：縦方向開口1Pa時[m^3/s]を長辺長さで割ったもの[m^3/s m]
- QH：横方向開口の1Pa時[m^3/s]を長辺長さで割ったもの[m^3/s m]
- SV：縦開口面積合計[m^2]
- SH：横開口面積[m^2]

In [ ]:
cal_QV(AI, SV, RAW) = AI * SV / 10.0^3 * ( 2.0 / RAW )^( 1/2 )
cal_QH(AI, SH, RAW) = AI * SH / 10.0^3 * ( 2.0 / RAW )^( 1/2 )

cal_QH (generic function with 1 method)


#### ※未完成：圧力を基準圧力9.8Paで割る理由がわからない。

基準圧力差$Q_o$を用いて換気量を計算する場合、

$Q = Q_o * (\frac{1}{g})^{\frac{1}{n}}*\frac{1}{3600}$  


入力値：9.8Pa時における流量[m3/h]を、1Pa時における[m3/s]に変換  
それを長辺の長さで割っている。※(式末尾の/ ( HU(i) - HD(i) )に相当。これを加えない記述もある）  


開口幅[mm]をを1Pa時における[m^3/s]を長辺の長さで割ったもの[m^2/s]に変換、RAWは２０℃における流量だったとおもう、、、


In [ ]:
cal_QOO( QOO, G, M ) = QOO * (1/G) ^ (1/M) / 3600.0
cal_QU( QU, G, M )   = QU * (1/G) ^ (1/M) / 3600.0

cal_QU (generic function with 1 method)

cal_QOO = [ QOO[i] * ( 1/G )^( 1/M[i] ) / 3600.0 / ( HU(i) - HD(i) ) for i = 1:length(M) ]  
cal_QU  = [ QU[i]  * ( 1/G )^( 1/MM[i]) / 3600.0 / B[i] for i = 1:length[MM] ]

## $\S 2.$ 実際の開口における換気量の計算

実際の開口における換気量を計算する場合、その計算方法はやや複雑である。  
その一因として、位置圧の存在がある。  
開口の上流側と下流側で温度に差がある場合、位置圧差は高さごとに異なる。  
そのため、高さごとに換気量が異なるため、高さを考慮した換気量の計算が必要となるのである。

<img src="picture/差圧の発生メカニズム.png" width="70%">

以下では、温度差の有無および中性帯の高さに応じた換気量の計算方法について示す。  

<img src="picture/中性帯ごとの分類.png" width="50%">

※風上法で計算を行う必要があるため、どちらの密度を採用するかの情報が必要となる点には留意すべき。

### $\S 2.1$ 温度差がないときor天井壁のとき

<img src="picture/隙間変数の名称.png" width="40%">

### $\S 2.1.1 $ 隙間における換気量

また、閉じた窓や扉などの隙間を想定すれば、同形状の隙間が複数存在する。  
そこで、縦方向の隙間における質量流量[kg/m<sup>3</sup>]を以下のように、

$$ SS = \rho q_v SH (\Delta p)^{1/MM} $$

横方向における隙間の流量を以下のように与える。

$$ SU = \rho q_h B (\Delta p)^{1/MM} $$  
$$ UC = \rho q_h B (\Delta p)^{1/MM} $$  

- SS : 縦開口流量[kg/s]
- SU,UC  : 上辺、下辺流量[kg/s]
- $\rho$ : 風上側空気の密度[kg/m<sup>3</sup>]
- $q_v, q_h$ : 縦方向、横方向における差圧1Pa時における通気量[m<sup>2</sup>/s]
- S：縦隙間本数[-]
- H：開口高さ[m]
- B：開口の幅[m]

なお、窓や扉のような開口を想定しているため横方向の隙間は上辺と下辺の2つのみとなっている。  
もし複雑な形状を扱う場合、$q_v, q_h$のいずれかを0とし、隙間情報を2つ以上に分けるか、別の関数を作成することを推奨する。

In [5]:
module flux_leakage_isothermal
    SU(RAW::Float64, QH::Float64, B::Float64, DP::Float64, MM::Float64)     =   RAW * QH * B * abs(DP)^(1/MM)
    SS(RAW::Float64, QV::Float64, S::Float64, H::Float64,  DP::Float64, M::Float64)   = RAW * QV * S * H * abs(DP)^(1/M)
    UC(RAW::Float64, QH::Float64, B::Float64, DP::Float64, MM::Float64)     =   RAW * QH * B * abs(DP)^(1/MM)
    
    function WU(RAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, H::Float64, DP::Float64, M::Float64, MM::Float64)
        return SU(RAW, QH, B, DP, MM) + SS(RAW, QV, S, H,  DP, M) + UC(RAW, QH, B, DP, MM)
    end
    
    # DPによる微分
    DSU(RAW::Float64, QH::Float64, B::Float64, DP::Float64, MM::Float64)   =   RAW * QH * B / MM * abs(DP) ^ ((1-MM)/MM)
    DSS(RAW::Float64, QV::Float64, S::Float64, H::Float64,  DP::Float64, M::Float64)  = RAW * QV * S * H / M * abs(DP) ^ ((1-M)/M)
    DUC(RAW::Float64, QH::Float64, B::Float64, DP::Float64, MM::Float64)   =   RAW * QH * B / MM * abs(DP) ^ ((1-MM)/MM)
    
    function DWU(RAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, H::Float64, DP::Float64, M::Float64, MM::Float64)
        return DSU(RAW, QH, B, DP, MM) + DSS(RAW, QV, S, H,  DP, M) + DUC(RAW, QH, B, DP, MM)
    end
end

Main.flux_leakage_isothermal

#### $\S 2.1.2$ 開口における換気量  

$$ WU = \alpha A \sqrt{2\rho}(\Delta p)^{\frac{1}{2}} $$  

- WU : 開口流量[kg/s]
- $\alpha$：流量係数   
- $A$：隙間面積[m<sup>2</sup>]
- $\rho$：空気の密度[kg/m<sup>3</sup>]
- $\Delta p$：室間の差圧[Pa]  

※通気量Q[m<sup>3</sup>/s]に密度を乗じている点に注意。

In [6]:
module flux_ventilation_isothermal
    WU( A::Float64, B::Float64, H::Float64, DP::Float64, RAW::Float64)   = A * B * H * (2/RAW)^(1/2) * abs(DP) ^ (1/2)
    DWU(A::Float64, B::Float64, H::Float64, DP::Float64, RAW::Float64)   = A * B * H * ( 2.0 * RAW )^0.5 / 2.0 * abs(DP) ^ ((1-2)/2)
end

Main.flux_ventilation_isothermal


### $\S 2.2$ 温度差があり、中立軸高さが上下端外にあるとき

#### $\S 2.2.1 $ 隙間における換気量

$$
    SS = \rho QH B (\Delta \rho g h_{nu})^{1/MM}\
$$  

$$
    SU = \rho S M Q_v  \bigl( \int_{0}^{H_{nu}} |{\Delta p}|^{\frac{1}{n}} dh - \int_{0}^{H_{nud}} |{\Delta p}|^{\frac{1}{n}} dh \bigl) 
$$

ただし、HNU = abs( HU - HN )であり、上端から中立軸高さまでの距離を示す。

ここで$\Delta p = \Delta \rho g h$とすると、

$$
    SU = \frac{ M }{ 1 + M } \rho  S  Q_V (\Delta \rho g)^{1/M}   | H_{nu}^{\frac{1+M}{M}} - H_{nd}^{\frac{1+M}{M}} |
$$

- SU : 上辺流量
- B：開口の幅
- S：縦隙間本数
- H：開口高さ
- M : 縦方向隙間特性値[-]  
- HN  = DP / ( DRAW * G ) #中立軸高さ
- HND = abs( HD - HN ) # 下端から中立軸高さまでの距離

In [7]:
module flux_leakage_noniso_uniflow

const G = 9.80665

function cal_SS(RAW::Float64, DRAW::Float64, QV::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64)
    return RAW * QV * S * M / ( 1.0+M ) * ( abs(DRAW)*G )^( 1.0/M ) * abs( HNU^( (1.0+M)/M ) - HND^( (1.0+M)/M) )
end

# 上辺と縦開口部分の隙間を通る流量。隙間と下辺の隙間で隙間特性値が分けてある。SU:[kg/s]
SU(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HNU::Float64, MM::Float64)  = RAW * QH * B * ( abs(DRAW) * G * HNU)^( 1.0/MM )
SS(RAW::Float64, DRAW::Float64, QV::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64)  = cal_SS(RAW, DRAW, QV, S, HNU, HND, M)

#下辺の隙間を通る流量
UC(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HND::Float64, MM::Float64) = RAW * QH * B * ( abs(DRAW) * G * HND )^( 1.0/MM )

# 枝の正味流量[kg/s]
function WU(RAW::Float64, DRAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64, MM::Float64)
    return SU(RAW, DRAW, QH, B, HNU, MM) + SS(RAW, DRAW, QV, S, HNU, HND, M) + UC(RAW, DRAW, QH, B, HND, MM)
end

# DPによる微分
DSU(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HNU::Float64, MM::Float64)   = RAW * QH * B * ( abs(DRAW ) * G * HNU )^( (1.0-MM ) / MM ) / MM
DSS(RAW::Float64, DRAW::Float64, QV::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64) = S * RAW * QV * ( abs(DRAW) * G )^( (1.0-M)/M ) * abs( HNU^( 1.0/M ) - HND^( 1.0/M ) )
DUC(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HND::Float64, MM::Float64)   = RAW * QH * B * ( abs(DRAW) * G * HND )^( ( 1.0-MM ) /MM ) / MM

function DWU(RAW::Float64, DRAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64, MM::Float64)
    return DSU(RAW, DRAW, QH, B, HNU, MM) + DSS(RAW, DRAW, QV, S, HNU, HND, M) + DUC(RAW, DRAW, QH, B, HND, MM)
end

end

Main.flux_leakage_noniso_uniflow

#### $\S 2.2.2$ 開口における換気量

$$
    Q = \alpha b \sqrt{\frac{2}{\rho}} \int_{h_{1}}^{h_{2}} | \Delta \rho g h|^{\frac{1}{2}} dh 
$$  

これを積分すると、以下のようになる。  

$$
    Q = \frac{2}{3} \alpha b \sqrt{2 \rho_i |\Delta \rho | g } \bigl| |h_{2}- h_{n}|^{ \frac{3}{2} } - |h_{1} - h_{n}|^{ \frac{3}{2} } \bigl| 
$$

In [8]:
module flux_ventilation_noniso_uniflow

const G = 9.80665

# 中立軸高さが上下端の外側にあるとき：式は同じだが密度の取り方（上流側か下流側か）が異なる
WU(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, HNU::Float64, HND::Float64) = 2.0 / 3.0 * A * B * (2.0 * RAW * abs(DRAW) * G )^0.5 * abs( HNU^1.5 - HND^1.5 )
#WD(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, HNU::Float64, HND::Float64) = 2.0 / 3.0 * A * B * (2.0 * RAW * abs(DRAW) * G )^0.5 * abs( HNU^1.5 - HND^1.5 )
DWU(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, HNU::Float64, HND::Float64)= A* B * (2.0 * RAW)^0.5 * ( abs(DRAW) * G )^(-0.5) * abs( HNU^0.5 - HND^0.5 )
#DWD(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, HNU::Float64, HND::Float64)= A* B * (2.0 * RAW)^0.5 * ( abs(DRAW) * G )^(-0.5) * abs( HNU^0.5 - HND^0.5 )

end

Main.flux_ventilation_noniso_uniflow

### $\S 2.3$温度差があり、中立軸高さが上下端の間にあるとき

#### $\S 2.3.1 $ 隙間における換気量

In [9]:
module flux_leakage_noniso_crossflow

const G = 9.80665

# 上辺と縦開口部（中性帯より上側）の隙間を通る流量。隙間と下辺の隙間で隙間特性値が分けてある。SU:[kg/s]
SU(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HNU::Float64, MM::Float64)     = RAW * QH * B * ( abs(DRAW) * G * HNU )^( 1.0/MM )
SSU(RAW::Float64, DRAW::Float64, QV::Float64, S::Float64, HNU::Float64, M::Float64) = RAW * QV * S * M * ( abs(DRAW) * G )^( 1.0/M ) * HNU^( ( 1.0+M ) / M ) / ( 1.0+M )
# 下辺と縦開口部（中性帯より下側）の隙間を通る流量
UC(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HND::Float64, MM::Float64) = RAW * QH * B * ( abs(DRAW) * G * HND )^( 1.0/MM )
SSD(RAW::Float64, DRAW::Float64, QV::Float64, S::Float64, HND::Float64, M::Float64) = RAW * QV * S * M * ( abs(DRAW) * G )^( 1.0/M ) * HND^( ( 1.0+M ) / M ) / ( 1.0+M )

# 枝の正味流量[kg/s]
function WU(RAW::Float64, DRAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64, MM::Float64)
    return SU(RAW, DRAW, QH, B, HNU, MM) + SSU(RAW, DRAW, QV, S, HNU, M)
end
function WD(RAW::Float64, DRAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64, MM::Float64)
    return UC(RAW, DRAW, QH, B, HND, MM) + SSD(RAW, DRAW, QV, S, HND, M)
end

# DPによる微分
DSU(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HNU::Float64, MM::Float64)   = RAW * QH * B * ( abs(DRAW ) * G * HNU )^( (1.0-MM ) / MM ) / MM
DSSU(RAW::Float64, DRAW::Float64, QV::Float64, S::Float64, HNU::Float64, M::Float64) = S * RAW * QV * ( abs(DRAW) * G )^( ( 1.0-M ) / M ) * HNU^( 1.0/M )
DUC(RAW::Float64, DRAW::Float64, QH::Float64, B::Float64, HND::Float64, MM::Float64)   = RAW * QH * B * ( abs(DRAW) * G * HND )^( ( 1.0-MM ) /MM ) / MM
DSSD(RAW::Float64, DRAW::Float64, QV::Float64, S::Float64, HND::Float64, M::Float64) = S * RAW * QV * ( abs(DRAW) * G )^( ( 1.0-M ) / M ) * HND^( 1.0/M )

function DWU(RAW::Float64, DRAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64, MM::Float64)
    return DSU(RAW, DRAW, QH, B, HNU, MM) + DSSU(RAW, DRAW, QV, S, HNU, M)
end
function DWD(RAW::Float64, DRAW::Float64, QH::Float64, QV::Float64, B::Float64, S::Float64, HNU::Float64, HND::Float64, M::Float64, MM::Float64)
    return DUC(RAW, DRAW, QH, B, HND, MM) + DSSD(RAW, DRAW, QV, S, HND, M)
end

end

Main.flux_leakage_noniso_crossflow

#### $\S 2.3.2$ 開口における換気量

In [10]:
module flux_ventilation_noniso_crossflow

const G = 9.80665

# 中立軸高さが上下端の内側にあるとき：
WU(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, Hight::Float64) = 2.0 / 3.0 * A * B * ( 2.0 * abs(DRAW) * G )^0.5 * RAW^0.5 * Hight^1.5
WD(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, Hight::Float64) = 2.0 / 3.0 * A * B * ( 2.0 * abs(DRAW) * G )^0.5 * RAW^0.5 * Hight^1.5
DWU(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, Hight::Float64)= A* B * 2.0^0.5 * ( abs(DRAW) * G )^(-0.5) * ( RAW * Hight )^0.5
DWD(A::Float64, B::Float64, DRAW::Float64, RAW::Float64, Hight::Float64)= A* B * 2.0^0.5 * ( abs(DRAW) * G )^(-0.5) * ( RAW * Hight )^0.5

end

Main.flux_ventilation_noniso_crossflow

<br>

## $\S3$ 圧力の計算方法

### $\S 3.1$ 起圧力[Pa]の計算  

#### 関数名（Fortran時）：SUBROUTINE PSCAL

床面圧力とは、床高さ密度差による圧力 - 外部風による圧力であり、室の床高さを考慮した圧力のことを指す。  
床面圧力は位置圧の差として以下のように表される。  

$$
    P_s = \rho_a g H_s
$$

なお空気の密度$\rho_a$は、状態方程式（ボイル-シャルルの法則）に従うものとし、基準温度（0℃）における密度を1.293[kg/m3]とすれば以下のように与えられる。  

$$
    \rho_a = \frac{353.25}{T}
$$  

- PS：起圧力[Pa] 外気の圧力を基準としている。
- HS：床面差(開口を挟んだ室同士の床面の高さの差)[m]
- WIN_V：風速[m/s]
- WC：風圧係数[-]

In [ ]:
function cal_Pressuremotive_force(temp::Float64, HS::Float64)
    rhoa = 353.25 / temp
    return rhoa * 9.80665 * HS
end

Main.zone_Ps

### $\S 3.2 $ 風圧力の計算  
※エース建築環境工学II 13章を参照のこと。  

<img src="picture/fuuatsu_keisu.png" width="70%">  

参照：エース建築環境工学II p.159 図13.4

一般に建物などの表面では外気風のもつ運動エネルギーは一部が静圧に変換される。  
この割合を風圧係数Cとよび、建物の形状、表面の位置、風向により異なる値をとる。  
建物から十分離れその影響を受けない風を自由風といい、この流速をV[m/s]とすると、壁面に働く風圧$p_w$は、  

$$
    p_w = C p_v = C \Bigl( \frac{\rho}{2}V^2 \Bigl)
$$  

ここで$p_w$：壁面に働く風圧[Pa]、$p_v$：自由風の動圧[Pa]である。  


In [1]:
# 風圧力の計算
function cal_wind_pressure_force(C::Float64, rho::Float64, v::Float64)
    return (1.0 / 2.0) * C * rho * v ^ 2.0
end
cal_wind_pressure_force(; C::Float64, rho::Float64, v::Float64) = cal_wind_pressure_force(C, rho, v)

cal_wind_pressure_force (generic function with 1 method)


なお、通常建物において風圧係数Cは、
- 風上側：0 <= C <=1
- 風下側：C <= 0

となる。ここで図13.4より風向による風圧係数を以下のようにモジュールとして定義する。

In [ ]:
# AfW：Angle from Wind direction
function cal_wind_pressure_coefficient( AfW::Float64 )
    if AfW < -180.0
        AfW = AfW + 360.0
    end
    # 0°~30°
    if ( -30.0 <= AfW && AfW <= 30  ) || ( 330.0 <= AfW )
        C = 0.75
    # 30°~75°
    elseif ( -75.0 <= AfW && AfW <= -30.0  )
        C = 1.25 + 0.75 / 45.0 * AfW
    elseif ( 30.0 <= AfW && AfW <= 75.0  )
        C = 1.25 - 0.75 / 45.0 * AfW
    elseif ( 285.0 <= AfW && AfW <= 330.0  )
        C = 1.25 + 0.75 / 45.0 * ( AfW - 360.0 )
    # 75°~90°
    elseif ( -90.0 <= AfW && AfW <= -75.0  )
        C = 2.0 + 0.4 / 15.0 * AfW
    elseif ( 75.0 <= AfW && AfW <= 90.0  )
        C = 2.0 - 0.4 / 15.0 * AfW
    elseif ( 270.0 <= AfW && AfW <= 285.0  )
        C = 2.0 + 0.4 / 15.0 * ( AfW - 360.0 )
    # 90°~105°
    elseif ( -105.0 <= AfW && AfW <= -90.0  )
        C = 0.2 + 0.1 / 15.0 * AfW
    elseif ( 90.0 <= AfW && AfW <= 105.0  )
        C = 0.2 - 0.1 / 15.0 * AfW
    elseif ( 255.0 <= AfW && AfW <= 270.0  )
        C = 0.2 + 0.1 / 15.0 * ( AfW - 360.0 )
    # 90°~105°
    elseif ( -180.0 <= AfW && AfW <= -105.0  ) || ( 105.0 <= AfW && AfW <= 255.0 )
        C = -0.5
    end
    return C
end

cal_wind_pressure_coefficient (generic function with 2 methods)

</br>  

## $\S 4$ 換気量計算

#### 入力変数名  

- ION：開口向き（VT：鉛直壁、HN：天井開口））  
※ 以前は0：鉛直壁、1：天井開口とされていた。  
- Type：開口状態（gap：隙間、opening：開口、constant：換気回数、fan：換気ファンなど） 
※ 以前はDという名称で、0：閉　１：開　2以上とされていた。
※ 天井壁またはD(I)＝５のとき（これが何さしてるかはわからない）A(I)＝0.5   開いているときまたは、D(I)＝２の時,A(I)＝0.7：A(I):流量係数）
- A : α：流量係数[-]
- B : 開口幅[mm]
- S : 縦開口本数
- HU : 開口上辺高さ[m]
- HD : 開口下辺高さ[m]
- M : 縦方向隙間特性値[-]
- MM : 横方向隙間特性値[-]
- RAW2_IP：枝上流側密度
- RAW2_IM：枝下流側密度
- DP：起圧力差
- DRAW：枝上流空気の密度-下流空気の密度
- QV : 縦方向開口1Pa時[m<sup>3</sup>/s]を長辺長さで割ったもの[m<sup>2</sup>/s]
- QH : 横方向開口の1Pa時[m<sup>3</sup>/s]を長辺長さで割ったもの[m<sup>2</sup>/s]

#### 出力変数
- W : 開口の正味流量
- WU : 開口正方向流量
- WD : 開口負方向流量
- DW : 開口[∂f/∂p]  
- DWU : 開口正方向[∂f/∂p]
- DWD : 開口負方向[∂f/∂p]

In [11]:
function cal_flux_ventilation( ION, Type, DP, DRAW, A, B, S, HU, HD, M, MM, RAW_IP, RAW_IM, QV, QH )
    
    # 重力加速度
    G = 9.80665

    # 開口の高さ＝開口上端高さー開口下端高さ
    H = HU - HD
    
    #####################################    
    #鉛直開口
    if ION == "VT"
        ############
        # 温度差なし
        if DRAW == 0.0

            # 仮定した起圧力と実際の気圧力との差がないとき
            if DP == 0.0
                WU  = 0.0
                WD  = 0.0
                DWU = 1.0e-5
                DWD = 1.0e-5

            # 起圧力差DPが正の時
            elseif DP > 0
                WD  = 0.0
                DWD = 0.0
                # 1) 隙間における漏気
                if Type == "gap"
                    WU  = flux_leakage_isothermal.WU(RAW_IP, QH, QV, B, S, H, DP, M, MM)
                    DWU = flux_leakage_isothermal.DWU(RAW_IP, QH, QV, B, S, H, DP, M, MM)
                # 2) 開口における漏気
                elseif Type == "opening"
                    WU  = flux_ventilation_isothermal.WU(A, B, H, DP, RAW_IP)
                    DWU = flux_ventilation_isothermal.DWU(A, B, H, DP, RAW_IP)
                end
            # 起圧力差DPが負の時
            elseif DP < 0
                WU  = 0.0
                DWU = 0.0
                # 1) 隙間における漏気
                if Type == "gap"
                    WD  = flux_leakage_isothermal.WU(RAW_IM, QH, QV, B, S, H, DP, M, MM)
                    DWD = flux_leakage_isothermal.DWU(RAW_IM, QH, QV, B, S, H, DP, M, MM)
                # 2) 開口における漏気
                elseif Type == "opening"
                    WD  = flux_ventilation_isothermal.WU(A, B, H, DP, RAW_IM)
                    DWD = flux_ventilation_isothermal.DWU(A, B, H, DP, RAW_IM)
                end
            end
            
        ############
        # 温度差が存在するとき
        else
            HN  = DP / ( DRAW * G ) #中立軸高さ
            HNU = abs( HU - HN ) # 上端から中立軸高さまでの距離
            HND = abs( HD - HN ) # 下端から中立軸高さまでの距離
            
            # 中立軸高さが上下端外にあるとき　＆　上流側？への流れのとき（上流・下流の向きは用件等）
            if (DRAW < 0 && HN <= HD) || ( DRAW > 0 && HN >= HU )
                WD  = 0.0
                DWD = 0.0
                # 1) 隙間における漏気
                if Type == "gap"
                    WU  = flux_leakage_noniso_uniflow.WU(RAW_IP, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    DWU = flux_leakage_noniso_uniflow.DWU(RAW_IP, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                # 2) 開口における漏気
                elseif Type == "opening"
                    WU  = flux_ventilation_noniso_uniflow.WU(A, B, DRAW, RAW_IP, HNU, HND)
                    DWU = flux_ventilation_noniso_uniflow.DWU(A, B, DRAW, RAW_IP, HNU, HND)
                end

            # 中立軸高さが上下端外にあるとき　＆　下流側？への流れのとき
            elseif ( DRAW < 0 && HN >= HU ) || ( DRAW > 0 && HN <= HD )
                WU  = 0.0
                DWU = 0.0
                # 1) 隙間における漏気
                if Type == "gap"
                    WD  = flux_leakage_noniso_uniflow.WU(RAW_IM, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    DWD = flux_leakage_noniso_uniflow.DWU(RAW_IM, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                # 2) 開口における漏気
                elseif Type == "opening"
                    WD  = flux_ventilation_noniso_uniflow.WU(A, B, DRAW, RAW_IM, HNU, HND)
                    DWD = flux_ventilation_noniso_uniflow.DWU(A, B, DRAW, RAW_IM, HNU, HND)
                end
            
            # 中立軸高さが上下端内にあるとき　＆　密度差が負のとき
            elseif DRAW < 0 && HD < HN && HN < HU 
                # 1) 隙間における漏気
                if Type == "gap"
                    WU  = flux_leakage_noniso_crossflow.WU( RAW_IP, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    WD  = flux_leakage_noniso_crossflow.WD( RAW_IM, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    DWU = flux_leakage_noniso_crossflow.DWU(RAW_IP, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    DWD = flux_leakage_noniso_crossflow.DWD(RAW_IM, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                # 2) 開口における漏気
                elseif Type == "opening"
                    WU  = flux_ventilation_noniso_crossflow.WU( A, B, DRAW, RAW_IP, HNU )
                    WD  = flux_ventilation_noniso_crossflow.WD( A, B, DRAW, RAW_IM, HND )
                    DWU = flux_ventilation_noniso_crossflow.DWU( A, B, DRAW, RAW_IP, HNU )
                    DWD = flux_ventilation_noniso_crossflow.DWD( A, B, DRAW, RAW_IM, HND )
                end
                
            # 中立軸高さが上下端内にあるとき　＆　密度差が負のとき
            elseif DRAW > 0 && HD < HN && HN < HU
                # 1) 隙間における漏気
                if Type == "gap"
                    WU  = flux_leakage_noniso_crossflow.WD( RAW_IP, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    WD  = flux_leakage_noniso_crossflow.WU( RAW_IM, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    DWU = flux_leakage_noniso_crossflow.DWD(RAW_IP, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                    DWD = flux_leakage_noniso_crossflow.DWU(RAW_IM, DRAW, QH, QV, B, S, HNU, HND, M, MM)
                # 2) 開口における漏気
                elseif Type == "opening"
                    WU  = flux_ventilation_noniso_crossflow.WU( A, B, DRAW, RAW_IP, HND )
                    WD  = flux_ventilation_noniso_crossflow.WD( A, B, DRAW, RAW_IM, HNU )
                    DWU = flux_ventilation_noniso_crossflow.DWU( A, B, DRAW, RAW_IP, HND )
                    DWD = flux_ventilation_noniso_crossflow.DWD( A, B, DRAW, RAW_IM, HNU )
                end
            end
        end

    elseif ION == 1
        println("まだ実装されてないよ")        
    end

    W   = WU - WD # Wを矢印の方向に合わせて、正負を変換している。
    DW  = DWU + DWD # 微分の計算
    
    return W, WU, WD, DW, DWU, DWD
end

cal_flux_ventilation(; ION, Type, DP, DRAW, A, B, S, HU, HD, M, MM, RAW_IP, RAW_IM, QV, QH) = cal_flux_ventilation( ION, Type, DP, DRAW, A, B, S, HU, HD, M, MM, RAW_IP, RAW_IM, QV, QH )

cal_flux_ventilation (generic function with 2 methods)

In [12]:
test_vf = cal_flux_ventilation( 
    ION = "VT", 
    Type= "gap", 
    DP  = 0.0, 
    DRAW= 0.1, 
    A   = 3.0, 
    B   = 1.0, 
    S   = 0.0, 
    HU  = 5.0, 
    HD  = 1.0, 
    M   = 2.0, 
    MM  = 2.0, 
    RAW_IP = 2.0, 
    RAW_IM = 2.0, 
    QV      = 1.0, 
    QH      = 1.0 )

(-6.409261176238541, 0.0, 6.409261176238541, 1.4614107443030637, 0.0, 1.4614107443030637)